In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from PIL import Image

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix
from sklearn.ensemble import AdaBoostClassifier
import seaborn as sns

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import sys
import os
import random
from pathlib import Path
import imageio
import skimage
import skimage.io
import skimage.transform
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly import tools
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
import scipy
from sklearn.model_selection import train_test_split
from sklearn import metrics
from keras import optimizers
from keras.initializers import Constant
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPool2D, Dropout, BatchNormalization,LeakyReLU
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping, ReduceLROnPlateau, LearningRateScheduler
from keras.utils import to_categorical
from keras.layers.advanced_activations import LeakyReLU, PReLU
import tensorflow_addons as tfa
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
import tensorflow as tf

In [ ]:
IMAGE_PATH = '../input/chinese-mnist/data/data/'
IMAGE_WIDTH = 64
IMAGE_HEIGHT = 64
IMAGE_CHANNELS = 1
RANDOM_STATE = 42
TEST_SIZE = 0.2
VAL_SIZE = 0.2
BATCH_SIZE = 32
NO_EPOCHS = 50
PATIENCE = 5
VERBOSE = 1

In [ ]:
dfc = pd.read_csv('/kaggle/input/chinese-mnist/chinese_mnist.csv')
images = np.array([np.asarray(Image.open('/kaggle/input/chinese-mnist/data/data/input_%d_%d_%d.jpg'%(x['suite_id'], x['sample_id'], x['code']))) for x in dfc.iloc])
pd.DataFrame(zip([9,10,100,1000,10000,100000000,0,1,2,3,4,5,6,7,8],dfc['character'].unique()), columns=['valor', 'caractere']).sort_values(by='valor')

In [ ]:
data_df = pd.read_csv('../input/chinese-mnist/chinese_mnist.csv')
print(data_df.shape) 
data_df.sample(100).head(4)

In [ ]:
image_files = list(os.listdir(IMAGE_PATH))
print("Number of image files: {}".format(len(image_files)))

In [ ]:
def create_file_name(x):
    
    file_name = f"input_{x[0]}_{x[1]}_{x[2]}.jpg"
    return file_name

In [ ]:
data_df["file"] = data_df.apply(create_file_name, axis=1)
data_df.head()

In [ ]:
file_names = list(data_df['file'])
print("Matching image names: {}".format(len(set(file_names).intersection(image_files))))

In [ ]:
def read_image_sizes(file_name):
    image = skimage.io.imread(IMAGE_PATH + file_name)
    return list(image.shape)

In [ ]:
tqdm.pandas()
m = np.stack(data_df['file'].progress_apply(read_image_sizes))
df = pd.DataFrame(m,columns=['w','h'])
data_df = pd.concat([data_df,df],axis=1, sort=False)
data_df.head()

In [ ]:
print(f"Number of suites: {data_df.suite_id.nunique()}")
print(f"Samples: {data_df.sample_id.unique()}")

In [ ]:
train_df, test_df = train_test_split(data_df, 
                                     test_size=TEST_SIZE, random_state=RANDOM_STATE, stratify=data_df["code"].values)

In [ ]:
train_df, val_df = train_test_split(train_df, 
                                    test_size=VAL_SIZE, random_state=RANDOM_STATE, stratify=train_df["code"].values)

In [ ]:
print("Train set rows: {}".format(train_df.shape[0]))
print("Test  set rows: {}".format(test_df.shape[0]))
print("Val   set rows: {}".format(val_df.shape[0]))

In [ ]:
def read_image(file_name):
    image = skimage.io.imread(IMAGE_PATH + file_name)
    image = skimage.transform.resize(image, (IMAGE_WIDTH, IMAGE_HEIGHT, 1), mode='reflect')
    return image[:,:,:]

In [ ]:
def categories_encoder(dataset, var='character'):
    X = np.stack(dataset['file'].apply(read_image))
    y = pd.get_dummies(dataset[var], drop_first=False)
    return X, y

In [ ]:
X_train, y_train = categories_encoder(train_df)
X_val, y_val = categories_encoder(val_df)
X_test, y_test = categories_encoder(test_df)

In [ ]:
model=Sequential()

model.add(Conv2D(filters = 64, kernel_size=(3,3), input_shape=(64, 64, 1), padding='same'))
model.add(PReLU(alpha_initializer=Constant(value=0.25)))
model.add(MaxPool2D(2))


model.add(Conv2D(filters = 128, kernel_size=(3,3), padding='same'))
model.add(PReLU(alpha_initializer=Constant(value=0.25)))
model.add(MaxPool2D(2))


model.add(Conv2D(filters = 160, kernel_size=(3,3),  padding='same'))
model.add(PReLU(alpha_initializer=Constant(value=0.25)))
model.add(MaxPool2D(2))


model.add(Conv2D(filters = 256, kernel_size=(3,3), padding='same'))
model.add(PReLU(alpha_initializer=Constant(value=0.25)))
model.add(Conv2D(filters = 256, kernel_size=(3,3), padding='same'))
model.add(PReLU(alpha_initializer=Constant(value=0.25)))
model.add(MaxPool2D(2))


model.add(Conv2D(filters = 384, kernel_size=(3,3),  padding='same'))
model.add(PReLU(alpha_initializer=Constant(value=0.25)))
model.add(Conv2D(filters = 384, kernel_size=(3,3), padding='same'))
model.add(PReLU(alpha_initializer=Constant(value=0.25)))
model.add(MaxPool2D(2))


model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(1024))
model.add(PReLU(alpha_initializer=Constant(value=0.25)))
model.add(Dropout(0.5))
model.add(Dense(15, activation='softmax'))
model.add(PReLU(alpha_initializer=Constant(value=0.25)))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

In [ ]:
annealer = LearningRateScheduler(lambda x: 1e-3 * 0.99 ** (x+NO_EPOCHS))
earlystopper = EarlyStopping(monitor='loss', patience=PATIENCE, verbose=VERBOSE)
checkpointer = ModelCheckpoint('best_model.h5',
                                monitor='val_accuracy',
                                verbose=VERBOSE,
                                save_best_only=True,
                                save_weights_only=True)

In [ ]:
train_model  = model.fit(X_train, y_train,
                  batch_size=BATCH_SIZE,
                  epochs=NO_EPOCHS,
                  verbose=1,
                  validation_data=(X_val, y_val),
                  callbacks=[earlystopper, checkpointer, annealer])

In [ ]:
def create_trace(x,y,ylabel,color):
        trace = go.Scatter(
            x = x,y = y,
            name=ylabel,
            marker=dict(color=color),
            mode = "markers+lines",
            text=x
        )
        return trace
    
def plot_accuracy_and_loss(train_model):
    hist = train_model.history
    acc = hist['accuracy']
    val_acc = hist['val_accuracy']
    loss = hist['loss']
    val_loss = hist['val_loss']
    epochs = list(range(1,len(acc)+1))
    #define the traces
    
    trace_ta = create_trace(epochs,acc,"Training accuracy", "Green")
    trace_va = create_trace(epochs,val_acc,"Validation accuracy", "Red")
    trace_tl = create_trace(epochs,loss,"Training loss", "Blue")
    trace_vl = create_trace(epochs,val_loss,"Validation loss", "Magenta")
    fig = tools.make_subplots(rows=1,cols=2, subplot_titles=('Training and validation accuracy',
                                                             'Training and validation loss'))
    #add traces to the figure
    fig.append_trace(trace_ta,1,1)
    fig.append_trace(trace_va,1,1)
    fig.append_trace(trace_tl,1,2)
    fig.append_trace(trace_vl,1,2)
    #set the layout for the figure
    fig['layout']['xaxis'].update(title = 'Epoch')
    fig['layout']['xaxis2'].update(title = 'Epoch')
    fig['layout']['yaxis'].update(title = 'Accuracy', range=[0,1])
    fig['layout']['yaxis2'].update(title = 'Loss', range=[0,1])
    #plot
    iplot(fig, filename='accuracy-loss')

plot_accuracy_and_loss(train_model)

In [ ]:
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
score = model.evaluate(X_val, y_val, verbose=0)
print('Val loss:', score[0])
print('Val accuracy:', score[1])

In [ ]:
def test_accuracy_report(model):
    predicted = model.predict(X_test)
    test_predicted = np.argmax(predicted, axis=1)
    test_truth = np.argmax(y_test.values, axis=1)
    print(metrics.classification_report(test_truth, test_predicted, target_names=y_test.columns)) 
    test_res = model.evaluate(X_test, y_test.values, verbose=0)
    print('Loss function: %s, accuracy:' % test_res[0], test_res[1])

In [ ]:
model_optimal = model
model_optimal.load_weights('best_model.h5')
score = model_optimal.evaluate(X_test, y_test, verbose=0)
print(f'Best validation loss: {score[0]}, accuracy: {score[1]}')

test_accuracy_report(model_optimal)